In [ ]:
from astropy.coordinates import SkyCoord
import numpy as np

import lvmguiding
import time
import matplotlib
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, 'lvmifusim/')
import IFU

my_ifu = IFU.IFU(25)

## Load Gaia Stars and convert their positions to focal plane coordinates

In [ ]:
coordinates_crowded = SkyCoord(270.48085943, -28.45945724,unit="deg")
coordinates_median = SkyCoord(195.10462204, -55.248677,unit="deg")
coordinates_empty = SkyCoord(184.18604651, 15.16853933,unit="deg")

In [ ]:
darks_coordinates = np.loadtxt("southern_darks.ascii")

In [ ]:
darks_coordinates.shape

In [ ]:
dark_index = 22

coordinates_dark = SkyCoord(darks_coordinates[dark_index,1],darks_coordinates[dark_index,2],unit="deg")

print(coordinates_dark)

In [ ]:
print(coordinates_crowded)

In [ ]:
culled_cat

In [ ]:
#dd_x_mm,dd_y_mm,culled_cat = lvmguiding.find_guide_stars(coordinates_crowded,pa=0,plotflag=False,return_focal_plane_coords=True)

c = coordinates_dark
custom_maglim = 17

maglim = custom_maglim
lvmguiding.mag_lim_lower = custom_maglim
time0 = time.time()
dd_x_mm,dd_y_mm,culled_cat = lvmguiding.find_guide_stars(c,pa=0,plotflag=False,return_focal_plane_coords=True,remote_catalog=False,remote_maglim=custom_maglim)
time1 = time.time()
print("Findin the stars in the focal plane within the Gaia catalog took {:.1f} s".format(time1-time0))
print()





print("Coordinates: ",c)
selection_mag = culled_cat["phot_g_mean_mag"] < maglim
print("Number of sources in focal plane: ",len(dd_x_mm))
print("Stars that pass mag selection: ",np.sum(selection_mag))

In [ ]:
# Define the IFU

my_ifu = IFU.IFU(24)

In [ ]:
fig,(ax1,ax2) = plt.subplots(figsize=(20,12),ncols=2)


lens_radii = 0.315/2 * np.ones_like(my_ifu.lensx) #This comes from PDR Document, Figure 4
contaminated = np.array(my_ifu.lensx) < -999
time0 = time.time()

for i in range(len(my_ifu.lensx)):
    current_x = my_ifu.lensx[i]
    current_y = my_ifu.lensy[i]
    current_r = lens_radii[i]
    
    d2 = (current_x-dd_x_mm[selection_mag])**2+(current_y-dd_y_mm[selection_mag])**2
    
    if d2.min() < current_r**2:
        #print("Star in fiber ",i)
        circle = plt.Circle((current_x,current_y),current_r,facecolor="r",alpha=0.5)
        ax2.add_patch(circle)
        contaminated[i]=True
#    my_circle = plt.Circle((my_ifu.lensx[index],my_ifu.lensy[index]),my_ifu.lensr[index],color="r")
#    ax1.add_patch(my_circle)
time1 = time.time()

print("Neighbour search took {:6.2}s".format(time1-time0))


fig.suptitle("Pointing Coordinates:\nRA: {:.6f}\nDEC: {:.6f}\n\nMag. Lim: {} gmag".format(c.ra,c.dec,maglim))
ax1.set_title("Full focal plane\nN Sources: {}".format(len(dd_x_mm)))


ax1.set_xlabel("Focal plane x [mm]")
ax1.set_ylabel("Focal plane y [mm]")
ax2.set_xlabel("Focal plane x [mm]")
ax2.set_ylabel("Focal plane y [mm]")

ax1.plot(dd_x_mm,dd_y_mm,"ko",ms=1)
ax1.plot(my_ifu.lensx,my_ifu.lensy,"bo",ms=1)
ax1.plot(np.array(my_ifu.lensx)[contaminated],np.array(my_ifu.lensy)[contaminated],"ro",ms=1)
ax1.set_aspect("equal")


ax2.set_xlim(-8,8)
ax2.set_ylim(-8,8)
ax2.set_aspect("equal")

patches = [plt.Circle(center, size) for center, size in zip(np.stack((my_ifu.lensx,my_ifu.lensy),axis=1),lens_radii)]

#
coll = matplotlib.collections.PatchCollection(patches, facecolors='none',edgecolor="b")
ax2.add_collection(coll)

ax2.plot(dd_x_mm,dd_y_mm,"ko",ms=2)
#ax2.plot(dd_x_mm[selection_mag],dd_y_mm[selection_mag],"ko",ms=4)


ax2.set_title("IFU\n{} of {} ({:.1f}%) fibers are contaminated with stars brighter {} gmag".format(np.sum(contaminated),len(contaminated),100*np.sum(contaminated)/len(contaminated),maglim))

circle = plt.Circle((-7,7),current_r,facecolor="none",edgecolor="b",alpha=0.5)
ax2.add_patch(circle)
circle = plt.Circle((-7,6.5),current_r,facecolor="r",alpha=0.5)
ax2.add_patch(circle)
ax2.text(-6.5,7,"Star-free Fiber".format(maglim))
ax2.text(-6.5,6.5,"Contaminated Fiber".format(maglim))






#fig.savefig("dark_patch_stellar_contamination.png",dpi=200,bbox_inches="tight",facecolor="w")

In [ ]:
color_array=["r","g","b","r","g","b"]


fig,ax = plt.subplots(figsize=(12,12))

for index,pa in enumerate([0,60,120,180,240,300]):
        print("PA: ",pa)
        if pa==0:
            culled_cat=lvmguiding.cat_full
        ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = lvmguiding.find_guide_stars(c,pa=pa,plotflag=False,recycled_cat=culled_cat)
        
        if pa==0:
            ax.plot(culled_cat["ra"],culled_cat["dec"],"k.",ms=1)
        ax.plot(ras,decs,".",c=color_array[index],label="PA = "+str(pa)+" deg")
        
ax.legend()

ax.set_xlabel("RA [deg]")
ax.set_ylabel("DEC [deg]")
ax.invert_xaxis()

## Save IFUs in a textfile for temporary stand alone version:

In [ ]:
output = np.stack((my_ifu.lensx,my_ifu.lensy,lens_radii),axis=1)

In [ ]:
output.shape

In [ ]:
np.savetxt("ifu_positions.xyr",output,fmt="%12.4f")

In [ ]:
https://wiki.sdss.org/pages/viewpage.action?spaceKey=LVM&title=GeoCoronal+Emission